# Category Prediction Layer

In [1]:
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

def cls_predictor(num_inputs, num_anchors, num_classes):
    return nn.Conv2d(num_inputs, num_anchors * (num_classes + 1),
                     kernel_size=3, padding=1)

# Bounding Box Prediction Layer

In [ ]:
def bbox_predictor(num_inputs, num_achors):
    return nn.Conv2d(num_inputs, num_anchors * 4, kernel_size=3, padding=1)